In [1]:
#function to test models 
import nest_asyncio
import warnings
from mb_utils.src.logging import logger

nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [2]:
from mb_pytorch.utils.version import version
version

'1.0.87'

In [3]:
from mb_pytorch.dataloader.loader import DataLoader

k = DataLoader('../scripts/detection/object_detection.yaml',logger=logger)
k_data = k.data_dict['data']
model_data_dict = k.data_dict['model']

Tue 04:23:48,687 INF Read data from yaml file: [{'data': {'file': {'root': '/home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dat


In [4]:
from mb_pytorch.training.train_params import train_helper
model_yaml_data = k.data_dict['model']
loss_attr,optimizer_attr,optimizer_dict,scheduler_attr,scheduler_dict = train_helper(model_yaml_data) 

In [5]:
from torch.utils.tensorboard import SummaryWriter
import os

path_logs = os.path.join(os.path.split(k_data['file']['root'])[0], 'logs')
writer = SummaryWriter(log_dir=path_logs)


In [6]:
train_loader,val_loader,train_dataset,test_dataset = k.data_load()


300row [00:00, 71624.04row/s]
Tue 04:23:49,644 INF Loaded dataframe from /home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv usin
Tue 04:23:49,647 INF Data file: {'root': '/home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv', 've
Tue 04:23:49,649 INF Data columns: Index(['image_id', 'image_path', 'bbox', 'label', 'image_type'], dtype='object')                  
Tue 04:23:49,650 INF Data will be split into train and validation according to train_file input : True                               
Tue 04:23:49,651 INF If unnamed columns are present, they will be removed.                                                           
Tue 04:23:49,652 INF If duplicate rows are present, they will be removed.                                                            
Tue 04:23:49,660 INF Length of data after removing invalid paths: 300                                                                
Tue 04:23:49,661 INF Skipping im

ValueError: malformed node or string on line 1: <ast.Name object at 0x79fb8936fa90>

In [7]:
train_dataset.__getitem__(0)

(tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ...,

In [8]:
test_dataset.__getitem__(4)

(tensor([[[0.2431, 0.2473, 0.2510,  ..., 0.3059, 0.2815, 0.2683],
          [0.2301, 0.2387, 0.2437,  ..., 0.3059, 0.2839, 0.2720],
          [0.2055, 0.2172, 0.2240,  ..., 0.3094, 0.2853, 0.2723],
          ...,
          [0.2452, 0.0142, 0.1149,  ..., 0.2067, 0.2212, 0.2017],
          [0.2719, 0.0462, 0.0008,  ..., 0.1792, 0.2091, 0.2041],
          [0.4482, 0.1613, 0.0118,  ..., 0.1353, 0.1759, 0.1720]],
 
         [[0.4566, 0.4669, 0.4804,  ..., 0.4510, 0.4241, 0.4095],
          [0.4391, 0.4523, 0.4658,  ..., 0.4510, 0.4264, 0.4132],
          [0.4107, 0.4269, 0.4421,  ..., 0.4545, 0.4346, 0.4239],
          ...,
          [0.4343, 0.1640, 0.2984,  ..., 0.3366, 0.3476, 0.3281],
          [0.4902, 0.2387, 0.1324,  ..., 0.3132, 0.3435, 0.3385],
          [0.6723, 0.3395, 0.1471,  ..., 0.2784, 0.3249, 0.3210]],
 
         [[0.3193, 0.3297, 0.3412,  ..., 0.3490, 0.3272, 0.3154],
          [0.3076, 0.3177, 0.3320,  ..., 0.3490, 0.3295, 0.3191],
          [0.2893, 0.3006, 0.3175,  ...,

In [9]:
for id, (image, target) in enumerate(train_loader):
    print(image.shape)
    print(target)
    print(id)
    break

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/mb_pytorch/dataloader/loader.py", line 383, in collate_fn
    'label': torch.tensor([b[1]['label'] for b in batch])} ## label is a list of labels or should be hot encoded?
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many dimensions 'str'


In [25]:
for batch_idx, (images, targets) in enumerate(train_loader):
    print("image is : {}".format(images.shape))
    images_l = list(image.to('cpu') for image in images) 
    print("target is : {}".format(targets))
    d = {}
    d['boxes'] = targets[1][:]
    d['labels'] = targets[0][:]
    targets = [{k: v.to('cpu') for k, v in t.items()} for t in [d]]
    #print("image_new is : {}".format(images_l))
    print("target_new is : {}".format(targets))
    break

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 316, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 173, in collate
    return [collate

In [10]:
targets[0]['labels']

NameError: name 'targets' is not defined

In [11]:
from mb_pytorch.models.modelloader import ModelLoader
model_data_load = ModelLoader(k.data_dict['model'])
model =  model_data_load.get_model()

Sat 00:42:44,904 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   


In [12]:
targets[0]

NameError: name 'targets' is not defined

In [13]:
targets[0]

NameError: name 'targets' is not defined

In [14]:
for target in targets:
    print(target['boxes'])
    break

NameError: name 'targets' is not defined

In [15]:
model(images_l,targets)

NameError: name 'images_l' is not defined

In [ ]:
images.shape

torch.Size([4, 3, 218, 178])

In [ ]:
#tuple(targets)
lk = [{k: v for k, v in t.items()} for t in tuple(targets)]
lk

[{'boxes': tensor([[ 95,  71, 226, 313],
          [ 72,  94, 221, 306],
          [216,  59,  91, 126],
          [622, 257, 564, 781]]),
  'labels': tensor([2880, 2937, 8692, 5805])}]

In [ ]:
target[i] if target[0] is not None else None

KeyError: 0

In [ ]:
len(images_l)

4

In [ ]:
boxes_list = [target['boxes'] for target in targets]
boxes_list

[tensor([[ 95,  71, 226, 313],
         [ 72,  94, 221, 306],
         [216,  59,  91, 126],
         [622, 257, 564, 781]])]

In [ ]:
boxes_label = [target['labels'] for target in targets]
boxes_label

[tensor([2880, 2937, 8692, 5805])]

In [ ]:
boxes_list[0][0]

tensor([ 95,  71, 226, 313])

In [ ]:
boxes_label[0][0]

tensor(2880)

In [ ]:
for i in range(len(boxes_list[0][0].tolist())):
    print(boxes_list[0][1].tolist()[i])


72
94
221
306


In [ ]:
boxes_list[0][1].tolist()

[72, 94, 221, 306]

In [ ]:
import torch
lk_new = []
for i in range(len(images_l)):
    temp_dict = {}
    # temp_l = []
    # temp_l_final = []
    # for i in range(len(boxes_list[0][0])):
    #     temp_l.append(int(boxes_list[0][i].tolist()[i]))
    #     temp_l_final.append(temp_l)
    temp_dict['boxes'] = torch.tensor([boxes_list[0][i].tolist()])
    temp_dict['labels'] = torch.tensor([boxes_label[0][i]])
    lk_new.append(temp_dict)
lk_new

[{'boxes': tensor([[ 95,  71, 226, 313]]), 'labels': tensor([2880])},
 {'boxes': tensor([[ 72,  94, 221, 306]]), 'labels': tensor([2937])},
 {'boxes': tensor([[216,  59,  91, 126]]), 'labels': tensor([8692])},
 {'boxes': tensor([[622, 257, 564, 781]]), 'labels': tensor([5805])}]

In [64]:
model(images_l,lk_new)

AssertionError: All bounding boxes should have positive height and width. Found invalid box [970.7864990234375, 264.9587097167969, 408.9887390136719, 565.8440551757812] for target at index 2.

In [16]:
from mb_pytorch.detection.training import detection_train_loop


In [17]:
detection_train_loop(k,model_data_dict,writer=writer,logger=logger)

Sat 00:43:06,477 INF Training loop Starting                                                                                          
Sat 00:43:06,906 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   
Sat 00:43:06,908 INF Model Loaded                                                                                                    
300row [00:00, 147479.04row/s]
Sat 00:43:06,914 INF Loaded dataframe from /home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv usin
Sat 00:43:06,916 INF Data file: {'root': '/home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv', 've
Sat 00:43:06,918 INF Data columns: Index(['image_id', 'image_path', 'bbox', 'label', 'image_type'], dtype='object')                  
Sat 00:43:06,920 INF Data will be split into train and validation according to train_file input : True                               
Sat 00:43:06,922 INF If unnamed

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 316, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 173, in collate
    return [collate